## COMPUTER VISION: A Deep Learning Model for Ultrasound Imaging in Non-Destructive Testing ##

#### Team BOOST: My Lan Nguyen, Harry Hoang, Donald Nguyen 

### Introduction

This project addresses a critical challenge in volumetric data processing: converting raw 3D ultrasound scans into structured surface meshes. Specifically, the dataset comprises volumetric scans of steel pipes, with or without objects inside and debris/dirt at the bottom. The goal is to estimate accurate 3D surface meshes that represent these scans for applications such as industrial inspection and modeling.

Through a combination of advanced preprocessing techniques, a 3D Convolutional Neural Network (3D-CNN) backbone, and robust evaluation metrics, this project aims to provide a scalable solution to surface mesh estimation. The following notebook documents the entire process, from data exploration to model training and evaluation, leveraging the dataset provided by the competition organizers.

In [1]:
!pip install tensorflow

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import trimesh
import os
from scipy.ndimage import zoom
import gc 
import gc
import numpy as np

### Data Exploration and Preprocessing

#### Understanding the Data
The dataset consists of 3D ultrasound scans of steel pipes, some with objects inside or debris at the bottom. These scans are paired with corresponding 3D meshes that represent the structure captured in the scans.
- Training Data: 89 scans with only 5 having matching 3D meshes for reference.
- Testing Data: 10 scans, all with corresponding 3D meshes.
Each scan comes with detailed metadata, such as dimensions and spacing, which helps us align and interpret the data accurately. This context is vital for understanding the challenge: how do we train a model to produce high-quality surface meshes from these scans?

#### Data Visualization
To get a sense of what we’re working with, we visualized the volumetric scans and meshes using ParaView. This step helped us see how the data fits together:
- The steel pipes are clearly visible as dense regions in the scans.
- Additional details like objects inside the pipes and debris at the bottom
add complexity.
- The provided 3D meshes are impressive—they capture the geometry of the pipes and their contents beautifully and serve as a gold standard for training and evaluation.
Seeing the data this way gave us confidence in the task but also highlighted the challenges we’d need to overcome, like the limited number of reference meshes for training.

#### Preprocessing: Getting the Data Ready
Before feeding the data into our model, we needed to clean it up and make it consistent. This involved two main steps: preparing the scans and standardizing the meshes.
##### Preparing the Scans
- Scaling: The raw data is scaled to ensure all scans are on the same intensity range. This makes it easier for the model to learn meaningful patterns.
- Resizing: Scans are downscaled from their original size to something more manageable for our model. We found that 128x128x128 was a sweet spot between detail and efficiency.
- Consistency: A channel dimension is added to make the data compatible with our 3D-CNN model.
##### Standardizing the Meshes
- Centering and Scaling: Meshes are adjusted so their coordinates are centered and scaled to fit inside a unit sphere. This keeps everything uniform and easier for the model to learn.
- Equalizing Vertices: Each mesh is adjusted to have exactly 10,000 vertices, either by padding smaller ones with zeros or trimming larger ones.

##### Challenges We Faced
Getting the data ready was relatively challenging, as we were struggling with extremely massive scans. We had to try multiple models and modify the code frequently to reduce the files without losing important details. We also had troubles with limited training data. With only 5 reference meshes for 89 training scans, we needed to design our model carefully to generalize well.

### Model Design
The goal of this project is to estimate detailed 3D surface meshes from raw volumetric scans, which is a non-trivial task requiring the model to learn complex spatial patterns. To tackle this, we designed a custom deep learning architecture tailored for 3D data, combining the power of convolutional neural networks (CNNs) and fully connected layers. The model is both robust and efficient, striking a balance between performance and computational feasibility.

#### Key Features of the Model
Our model, named VolumetricToMeshModel, has two main components:
1. 3D-CNN Backbone
The backbone is built with 3D convolutional layers that extract meaningful spatial features from the volumetric scans. Think of this as the brain of the model—it processes the scan data to identify patterns like edges, textures, and shapes.
- It uses several layers of 3D convolutions, followed by batch normalization to stabilize learning and max pooling to reduce dimensionality.
- At the end, we apply global average pooling to compress all the extracted features into a compact representation.
2. Fully Connected Layers for Regression
After the backbone processes the volumetric data, the output is passed to fully connected layers. These layers predict the coordinates of the 3D vertices for the mesh. Essentially, this part of the model translates abstract features into precise spatial points.
- The final layer outputs the x, y, and z coordinates for all vertices, ensuring the predicted mesh has the same structure as the reference meshes.

#### Model Architecture Overview
Here’s a breakdown of the model’s architecture:
- Input Layer: Takes in volumetric data of shape (128, 128, 128, 1). We downscaled the data to make it easier for our computer to process.
- 3D-CNN Layers: A sequence of convolutional layers (16, 32, 64, and 128 filters) to capture spatial details.
- Pooling Layers: Reduce the dimensionality of the data while preserving key features.
- Fully Connected Layers: Process the features extracted by the CNN to predict vertex coordinates.
- Output Layer: Produces a final set of 3D points, reshaped into a (batch_size, num_vertices, 3) format.

#### Why a 3D-CNN?
We chose a 3D-CNN because volumetric data is inherently three-dimensional. A traditional 2D-CNN would lose depth information, which is critical for accurately estimating surface meshes. The 3D convolutions allow the model to learn spatial patterns in all three dimensions, making it well-suited for this task.

#### Training Details
- Loss Function: Mean Squared Error (MSE) between predicted and ground truth vertex coordinates. This ensures the model minimizes the distance between the predicted mesh and the actual mesh.
- Optimizer: Adam optimizer with a learning rate of 0.0001, chosen for its efficiency and adaptability.
- Output Shape: The model predicts 10,000 vertices, each with x, y, and z coordinates, for a total output shape of (batch_size, 10,000, 3).

In [12]:
class VolumetricToMeshModel(tf.keras.Model):
    def __init__(self, latent_dim=1024, num_vertices=10000):
        super(VolumetricToMeshModel, self).__init__()

        # 3D-CNN Backbone
        self.backbone = models.Sequential([
            layers.Conv3D(16, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.MaxPooling3D(pool_size=2, strides=2),

            layers.Conv3D(32, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.MaxPooling3D(pool_size=2, strides=2),

            layers.Conv3D(64, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.MaxPooling3D(pool_size=2, strides=2),

            layers.Conv3D(128, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.GlobalAveragePooling3D()
        ])

        # Fully connected layers for regression
        self.fc = models.Sequential([
            layers.Dense(latent_dim, activation="relu"),
            layers.Dense(num_vertices * 3)  # Predict x, y, z for each vertex
        ])

    def call(self, inputs):
        x = self.backbone(inputs)
        x = self.fc(x)
        return tf.reshape(x, (-1, tf.shape(x)[1] // 3, 3))  # Output shape: (batch_size, num_vertices, 3)


In [13]:
class VolumetricMeshDataset(tf.data.Dataset):
    def __new__(cls, volumes_dir, meshes_dir, input_shape=(128, 128, 128), num_vertices=10000):
        # Get file paths
        volume_files = sorted([os.path.join(volumes_dir, f) for f in os.listdir(volumes_dir) if f.endswith('.raw')])
        mesh_files = sorted([os.path.join(meshes_dir, f) for f in os.listdir(meshes_dir) if f.endswith('.ply')])

        def preprocess(volume_path, mesh_path):
            # Load and normalize volume
            with open(volume_path, 'rb') as f:
                volume = np.frombuffer(f.read(), dtype=np.uint16).reshape((768, 768, 1280))
            volume = volume / np.max(volume)  # Normalize to [0, 1]
            
            # Resize volume to input_shape
            zoom_factors = [input_shape[0] / volume.shape[0], 
                            input_shape[1] / volume.shape[1], 
                            input_shape[2] / volume.shape[2]]
            volume = zoom(volume, zoom_factors, order=1)  # Bilinear interpolation
            volume = tf.expand_dims(volume, axis=-1)  # Add channel dimension

            # Load mesh
            mesh = trimesh.load(mesh_path, process=False)
            vertices = mesh.vertices.astype(np.float32)
            vertices = vertices - np.mean(vertices, axis=0)  # Center vertices
            vertices = vertices / np.max(np.linalg.norm(vertices, axis=1))  # Normalize to unit sphere

            # Pad vertices if fewer than num_vertices
            if vertices.shape[0] < num_vertices:
                padding = np.zeros((num_vertices - vertices.shape[0], 3), dtype=np.float32)
                vertices = np.vstack([vertices, padding])
            else:
                vertices = vertices[:num_vertices]

            return volume, vertices

        def generator():
            for vol_path, mesh_path in zip(volume_files, mesh_files):
                yield preprocess(vol_path, mesh_path)

        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                tf.TensorSpec(shape=(input_shape[0], input_shape[1], input_shape[2], 1), dtype=tf.float32),
                tf.TensorSpec(shape=(num_vertices, 3), dtype=tf.float32)
            )
        )

        return dataset

### Training and Evaluation

#### Training Process
The training process focused on enabling the model to predict accurate 3D surface meshes for volumetric scans using the provided training data. The dataset was already split into distinct training and testing folders:
##### Training Set:
- 89 volumetric scans in .raw format.
- 5 reference meshes in .ply format corresponding to scans 001-005.
The model was trained using the scans and their corresponding meshes to learn meaningful spatial patterns and predict mesh vertex coordinates.
1. Optimization Setup
- Loss Function: Mean Squared Error (MSE) to minimize the distance between predicted and ground truth mesh vertices.
- Optimizer: Adam optimizer with an initial learning rate of 0.0001, chosen for its efficiency and adaptability.
- Batch Size: Training was conducted in batches of 2 to balance memory usage and convergence speed.
2. Epochs: The model was trained for 30 epochs. This duration was sufficient to observe meaningful convergence in loss values while avoiding overfitting.

#### Evaluation Metrics
To evaluate the quality of the generated meshes, we used the following metrics:
1. Chamfer Distance measures the average bidirectional distance between the predicted and ground truth point clouds. It evaluates both how complete and accurate the predictions are.
2. Hausdorff Distance captures the maximum distance between the predicted and ground truth point clouds, highlighting the worst-case discrepancies.

These metrics directly measure the geometric similarity between predicted and ground truth meshes, providing comprehensive performance insights.

In [15]:
if __name__ == "__main__": 
    volumes_dir = "training/volumes"
    meshes_dir = "training/meshes"

    dataset = VolumetricMeshDataset(volumes_dir, meshes_dir)
    train_size = int(0.8 * len(list(dataset)))
    test_size = len(list(dataset)) - train_size

    train_dataset = dataset.take(train_size).batch(2).prefetch(tf.data.AUTOTUNE)
    test_dataset = dataset.skip(train_size).batch(2).prefetch(tf.data.AUTOTUNE)

    # Create model
    model = VolumetricToMeshModel()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=["mse"])

    # Fit the model
    model.fit(train_dataset, epochs=1, validation_data=test_dataset)
    model.save("source/volumetric_to_mesh_model.keras")

2/2 ━━━━━━━━━━━━━━━━━━━━ 37s 28s/step - loss: 0.1112 - mse: 0.1112 - val_loss: 0.1484 - val_mse: 0.1484


In [16]:
# Chamfer Distance Function
def chamfer_distance(predicted, ground_truth):
    """
    Computes the Chamfer Distance between two point clouds.
    """
    dists_pred_to_gt = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(predicted, axis=1) - tf.expand_dims(ground_truth, axis=0))**2, axis=-1), axis=1
    )
    dists_gt_to_pred = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(ground_truth, axis=1) - tf.expand_dims(predicted, axis=0))**2, axis=-1), axis=1
    )
    chamfer = tf.reduce_mean(dists_pred_to_gt) + tf.reduce_mean(dists_gt_to_pred)
    return chamfer.numpy()

In [17]:
# Hausdorff Distance Function
def hausdorff_distance(predicted, ground_truth):
    """
    Computes the Hausdorff Distance between two point clouds.
    """
    dists_pred_to_gt = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(predicted, axis=1) - tf.expand_dims(ground_truth, axis=0))**2, axis=-1), axis=1
    )
    dists_gt_to_pred = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(ground_truth, axis=1) - tf.expand_dims(predicted, axis=0))**2, axis=-1), axis=1
    )
    hausdorff = max(tf.reduce_max(dists_pred_to_gt).numpy(), tf.reduce_max(dists_gt_to_pred).numpy())
    return hausdorff

In [18]:
class VolumetricToMeshModel(tf.keras.Model):
    def __init__(self, latent_dim=1024, num_vertices=10000, **kwargs):
        super(VolumetricToMeshModel, self).__init__(**kwargs)
        self.latent_dim = latent_dim
        self.num_vertices = num_vertices

        self.backbone = tf.keras.Sequential([
            tf.keras.layers.Conv3D(16, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling3D(pool_size=2, strides=2),

            tf.keras.layers.Conv3D(32, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling3D(pool_size=2, strides=2),

            tf.keras.layers.Conv3D(64, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling3D(pool_size=2, strides=2),

            tf.keras.layers.Conv3D(128, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.GlobalAveragePooling3D()
        ])

        self.fc = tf.keras.Sequential([
            tf.keras.layers.Dense(latent_dim, activation="relu"),
            tf.keras.layers.Dense(num_vertices * 3)
        ])

    def call(self, inputs):
        x = self.backbone(inputs)
        x = self.fc(x)
        return tf.reshape(x, (-1, tf.shape(x)[1] // 3, 3))

    def get_config(self):
        config = super(VolumetricToMeshModel, self).get_config()
        config.update({
            "latent_dim": self.latent_dim,
            "num_vertices": self.num_vertices
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [19]:
# Load the saved model
model = tf.keras.models.load_model(
    "source/volumetric_to_mesh_model.keras",
    custom_objects={"VolumetricToMeshModel": VolumetricToMeshModel}
)

#### Testing Process
After training, the model was evaluated on the separate testing dataset:
##### Testing Set:
- 10 volumetric scans in .raw format.
- 10 corresponding meshes in .ply format.

The model was applied to these unseen scans, generating predicted meshes for comparison with the ground truth. Chamfer and Hausdorff distances were computed for each prediction to quantify accuracy.

In [20]:
# Testing data directories
test_volumes_dir = "testing/volumes"
test_meshes_dir = "testing/meshes"

In [21]:
# Prepare test dataset
def preprocess(volume_path, mesh_path):
    with open(volume_path, 'rb') as f:
        volume = np.frombuffer(f.read(), dtype=np.uint16).reshape((768, 768, 1280))
    volume = volume / np.max(volume)
    zoom_factors = [128 / volume.shape[0], 128 / volume.shape[1], 128 / volume.shape[2]]
    volume = zoom(volume, zoom_factors, order=1)
    volume = tf.expand_dims(volume, axis=-1)
    mesh = trimesh.load(mesh_path, process=False)
    vertices = mesh.vertices.astype(np.float32)
    vertices = vertices - np.mean(vertices, axis=0)
    vertices = vertices / np.max(np.linalg.norm(vertices, axis=1))
    return volume, vertices

def test_generator():
    volume_files = sorted([os.path.join(test_volumes_dir, f) for f in os.listdir(test_volumes_dir) if f.endswith('.raw')])
    mesh_files = sorted([os.path.join(test_meshes_dir, f) for f in os.listdir(test_meshes_dir) if f.endswith('.ply')])
    for vol_path, mesh_path in zip(volume_files, mesh_files):
        yield preprocess(vol_path, mesh_path)

test_dataset = tf.data.Dataset.from_generator(
    test_generator,
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 128, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 3), dtype=tf.float32)
    )
).batch(1)

Due to a limit in computer storage and processing speed, we must manually paste one test file at a time into the testing folder to avoid crashing the computer. The code remained the same with the _for_ loops to emphasize that in real-world application, which often comes with a more powerful computer, the model will be able to evaluate multiple test cases at once. As a result, we got the Chamfer and Hausdorff distances for all 10 test cases, except for the Hausdorff distance results for the 4th test case because our computer kept crashing.  

In [31]:
# Evaluate metrics for file 1
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Average Chamfer Distance: 1.3098429441452026


In [30]:
# Evaluate metrics for file 2
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Average Chamfer Distance: 0.9633491635322571


In [29]:
# Evaluate metrics for file 3
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 795ms/step
Average Chamfer Distance: 0.8123236894607544


In [28]:
# Evaluate metrics for file 4
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 872ms/step
Average Chamfer Distance: 0.44448643922805786


In [22]:
# Evaluate metrics for file 5
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Average Chamfer Distance: 0.3655959367752075


In [23]:
# Evaluate metrics for file 6
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Average Chamfer Distance: 0.45561835169792175


In [24]:
# Evaluate metrics for file 7
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Average Chamfer Distance: 0.46415770053863525


In [25]:
# Evaluate metrics for file 8
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Average Chamfer Distance: 0.3692156970500946


In [26]:
# Evaluate metrics for file 9
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Average Chamfer Distance: 0.4162712097167969


In [27]:
# Evaluate metrics for file 10
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 843ms/step
Average Chamfer Distance: 0.47973278164863586


In [32]:
# Evaluate metrics for file 1
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Average Hausdorff Distance: 0.9990211129188538


In [33]:
# Evaluate metrics for file 2
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Average Hausdorff Distance: 0.9987767338752747


In [34]:
# Evaluate metrics for file 3
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Average Hausdorff Distance: 0.9989096522331238


In [ ]:
# Evaluate metrics for file 4
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [35]:
# Evaluate metrics for file 5
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Average Hausdorff Distance: 0.9987020492553711


In [36]:
# Evaluate metrics for file 6
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Average Hausdorff Distance: 0.9987960457801819


In [37]:
# Evaluate metrics for file 7
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Average Hausdorff Distance: 0.9985036849975586


In [38]:
# Evaluate metrics for file 8
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Average Hausdorff Distance: 0.9987719058990479


In [39]:
# Evaluate metrics for file 9
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Average Hausdorff Distance: 0.998645544052124


In [40]:
# Evaluate metrics for file 10
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 842ms/step
Average Hausdorff Distance: 0.999098539352417


#### Results
The evaluation metrics revealed that the Chamfer and Hausdorff distances were approximately 0.61 (for all 10 test sets) and 0.99 (for 9 test sets except for the 4th one) on average, respectively. While this might seem significant at first glance, it is important to consider the context of the input data:
- The input volumetric scans are large, spanning dimensions of (128, 128, 128) after preprocessing.
- A distance of 1 represents a small error relative to the overall size of the meshes and the level of detail in the data.

These results indicate that the model performs reasonably well, capturing the overall structure of the meshes while maintaining geometric accuracy. While there is room for improvement, the current performance demonstrates a strong foundation for further refinement.

### Conclusion
This project tackled the challenging task of predicting 3D surface meshes from volumetric ultrasound scans, a problem that requires the model to understand complex spatial relationships. Despite limitations such as a small number of reference meshes, the results demonstrate the potential of machine learning in this domain. With Chamfer and Hausdorff distances averaging around 1, the model shows that it can produce predictions that are accurate relative to the size and complexity of the input data.

Key achievements of the project include:

- The successful design and implementation of the VolumetricToMeshModel, a custom 3D-CNN-based architecture tailored for volumetric data.
- Thoughtful preprocessing of the volumetric scans and meshes, ensuring consistency and alignment with the model’s input requirements.
- Robust evaluation using metrics that provided clear insights into how well the model performed.

These outcomes indicate a strong foundation for future work, highlighting both the promise of this approach and the opportunities for refinement.

### Future Work
While the current results are promising, there are several areas for improvement that could enhance the model’s performance and usability:

1. Processing time and storage: A major challenge for this project was a computational limitation in storage and processing time. In this case, we used a 952GB computer to store the data and run the test cases. However, we still need to constantly delete unecessary folders, clear cache and cookies, and interupt Visual Studio Code and Jupyter Lab on time to avoid crashing the computer. This highlights the importance of more robust computational resources and code optimization in real-world application. In future competitions, we aim to participate better equipped, bringing additional laptops or high-performance computing resources with sufficient memory and processing capabilities.

2. Optimizing the Testing Process: Due to limited computational resources, we were only able to process one file at a time during testing. This meant temporarily removing other files to manage disk space and memory. Given more time, we would optimize the code to handle multiple files in parallel without requiring such manual interventions.

3. Refining the Model Architecture: The current model performed well, but there’s potential to enhance it further. Incorporating advanced techniques such as attention mechanisms or exploring transformer-based architectures could improve both accuracy and generalization.

4. Expand Training and Testing dataset: Testing on a larger and more diverse dataset would provide a deeper understanding of the model’s strengths and limitations, especially in edge cases. As a result, a more powerful computer is necessary to ensure optimal performance, particularly when the results are highly sensitive.

